In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"  
os.environ['NORMALIZATION_TYPE'] = 'zero_unit_variance'  

import sys
sys.path.append("..")
sys.path.append("../mango")
sys.path.append("../mango/design_baselines/diff")


import numpy as np
import torch
import pytorch_lightning as pl

import mango_jupyter_scripts
import design_bench
from helpers import seed_everything 

from SDE import DiffusionSOO



In [12]:
#!/usr/bin/env python3
"""
Main driver program for the mango as one of baseline methods for design-bench.

"""
import argparse
import logging
def build_args() -> argparse.Namespace:
    """
    Defines the experimental arguments for offline MBO baseline method
    evaluation.
    Input:
        None.
    Returns:
        A namespace containing the experimental argument values.
    """
    parser = argparse.ArgumentParser(description="Our method Experiments")

    parser.add_argument(
        "--task",
        type=str,
        default='Superconductor-RandomForest-v0', 
        choices=[task.task_name for task in design_bench.registry.all()],
        help="The name of the design-bench task for the experiment."
    )
    parser.add_argument(
        "--logging-dir",
        type=str,
        default=None,
        help="Logging directory to save optimization results to."
    )
    parser.add_argument(
        "--budget", type=int, default=2048, help="Query budget. Default 2048."
    )
    parser.add_argument(
        "--hidden-size",
        type=int,
        default=2048,
        help="Hidden size of the surrogate objective model. Default 2048."
    )
    parser.add_argument(
        "--seed", type=int, default=2024, help="Random seed. Default 42."
    )
    parser.add_argument(
        "--device", type=str, default="auto", help="Device. Default `auto`."
    )
    args = parser.parse_args([])
    return args

args = build_args()

In [13]:
from mango_jupyter_scripts.training import train_surrogate, mango_train
# Training a surrogate forward model (It is merely to provide a forward model for comparison purposes only).
forwardmodel = train_surrogate(task_name=args.task, hidden_size=args.hidden_size)

Loading model from ../checkpoints/surrogate/Superconductor-RandomForest-v0-surrogate-100epoch.ckpt


In [14]:
dataset_kwargs=dict(
    max_samples=10000,
    distribution=None,
    max_percentile=80,
    min_percentile=0) 

task = design_bench.make( args.task, dataset_kwargs=dataset_kwargs)
normalize_x = False
normalize_y = True
x_min, x_max, y_min, y_max = np.min(task.dataset.x, axis=0), np.max(task.dataset.x, axis=0), np.min(task.dataset.y, axis=0), np.max(task.dataset.y, axis=0) 
try:
    x1_range = task.dataset.x1_range
    x2_range = task.dataset.x2_range
    x_lbound = np.array([x1_range[0], x2_range[0]])
    x_ubound = np.array([x1_range[1], x2_range[1]])
except:
    x_lbound = task.oracle.internal_dataset.x.min(axis=0)
    x_ubound = task.oracle.internal_dataset.x.max(axis=0)
    y_lbound = task.oracle.internal_dataset.y.min()
    y_ubound = task.oracle.internal_dataset.y.max()
    
print("##################Before Normalization##################")
print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

if normalize_y:
    task.map_normalize_y()
    y_min = task.normalize_y(y_min)[0]
    y_max = task.normalize_y(y_max)[0]
    y_lbound = task.normalize_y(y_lbound)[0]
    y_ubound = task.normalize_y(y_ubound)[0]
if normalize_x:
    task.map_normalize_x()
    x_min = task.normalize_x(x_min)[0]
    x_max = task.normalize_x(x_max)[0]
    x_lbound = task.normalize_x(x_lbound)[0]
    x_ubound = task.normalize_x(x_ubound)[0]

if normalize_x or normalize_y:
    print("##################After Normalization##################")
    print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
    print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
    print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

##################Before Normalization##################
x_min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 x_max: [ 14.        0.        3.       40.      105.       97.2       5.
  50.        4.        0.        4.       12.       99.925   100.
  20.       15.        3.        0.        3.3      14.        5.
  75.       79.5       3.        5.       30.       35.38     45.
  98.       20.       41.       42.       18.       19.        5.
   0.        4.       11.        9.       96.71     99.445    99.992
   3.       64.       45.       50.99745   7.        2.       25.
  97.       75.       66.6       4.        0.        3.       24.
  98.        4.        4.        2.        0.        4.        6.
   4.        5.        3.        5.        4.41      4.       16.
 

In [15]:
import shutil
args = build_args()
 
num_epochs=  1000

args.logging_dir = args.logging_dir or f"../logs/run_ours_{args.task}_seed{args.seed}_budget{args.budget}_hidden{args.hidden_size}_epoch{num_epochs}_betamax{0.05}_01normalization"
print(args.logging_dir )

logging_dir = args.logging_dir
version = 1
while os.path.exists(logging_dir):
    logging_dir = f"{args.logging_dir}_v{version}"
    version += 1
os.makedirs(logging_dir, exist_ok=True)

seed_everything(args.seed)
torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore", message="The dataloader, .* does not have many workers.*")

logging.basicConfig(
    level=logging.INFO,  # 设置最低日志级别
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # 设置日志格式
    handlers=[
        logging.FileHandler(f'{args.logging_dir}/running.log'),  # 将日志写入文件
        # logging.StreamHandler()  # 同时将日志输出到控制台
    ]
)

try:
    # # 尝试访问 inverse_model
    inverse_model

except NameError:
    inverse_model = None  # 如果未定义，设置为 None

 

# inverse_model = mango_train(
#     task=task,
#     task_name=args.task,
#     ckpt_dir=args.logging_dir,
#     hidden_size=args.hidden_size,
#     device=args.device,
#     seed=args.seed,
#     condition_training=False,
#     num_epochs=num_epochs,
#     dropout=0.0,
#     augment=True,
#     lr=1e-4,
#     batch_size=64,
#     debais=True,
# )


Global seed set to 2024


../logs/run_ours_Superconductor-RandomForest-v0_seed2024_budget2048_hidden2048_epoch1000_betamax0.05_01normalization


In [16]:

model_ckpt = f"/home/tzhouaq/ManGO_SOO/logs/run_ours_Superconductor-RandomForest-v0_seed2024_budget2048_hidden2048_epoch1000_betamax0.05_01normalization/ours-superconductor/superconductor-seed2024-hidden2048-score_matchingFalse.ckpt"
inverse_model = DiffusionSOO.load_from_checkpoint(
    model_ckpt,
    taskname=task.dataset_name,
    task=task,
    lr=1e-4,
    dropout_p=0.0,
    hidden_size=args.hidden_size,
    debais=True,
    augment=True,
    beta_min=0.01,
    beta_max=2,
    condition_training=False
)
# inverse_model.gen_sde.forwardmodel.load_state_dict(model.state_dict())
# inverse_model.forwardmodel.load_state_dict(model.state_dict())


uncondition_training T: {1.0}


In [17]:
from mango_jupyter_scripts.eval_fks import mango_eval
# This cell is to conduct unconditional generation

augment = True
condition_training = False
 
if augment:
    if not normalize_y:
        y_lbound = [y_lbound]
        y_ubound = [y_ubound]
    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

# y_given_list = np.linspace(0.5, 15, 200)
print('num_samples budget:', args.budget)
num_samples = args.budget
 
 
# y_given = torch.tensor([2])
res = mango_eval(
    task=task,
    task_name=args.task,
    forwardmodel=forwardmodel,
    inverse_model=inverse_model,
    device='auto',
    logging_dir=args.logging_dir,
    num_samples=num_samples,
    augment=augment,
    clip_dic=clip_dic,
    # gamma=0,
    condition_training=condition_training,
    num_steps=5,
    seed=args.seed,
 
)

res['x'].shape#, res['y'].shape, res['y_scores'].shape
 

{'simple_clip': True, 'clip_min': tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9925]), 'clip_max': tensor([ 14.0000,   0.0000,   3.0000,  40.0000, 105.0000, 120.0000,  12.8000,
   

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


(2048, 86)

In [18]:

import numpy as np

# designs = np.load(os.path.join(args.logging_dir, "solution.npy"))
# preds = np.load(os.path.join(args.logging_dir, "predictions.npy"))
# scores = np.load(os.path.join(args.logging_dir, "scores.npy"))
y_unob_max = 185#task.oracle.internal_dataset.y.max()
y_unob_min = task.oracle.internal_dataset.y.min()

designs = res['x']
preds =   res['y_gen']
# preds = res['y']
scores = res['y_scores']

# designs = designs.reshape(1, -1, designs.shape[-1]) 
# preds = preds.reshape(1, -1, preds.shape[-1])
# scores = scores.reshape(1, -1, scores.shape[-1])
print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

  
# task = design_bench.make(args.task)
if task.is_normalized_y:
    best_training_score = task.denormalize_y(np.max(task.dataset.y))#[0]
else:
    best_training_score = np.max(task.dataset.y)
print("Training dataset:\n The best score during training is: ",best_training_score )
oracle = np.max(scores)
print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
print(f' The corresponding design is: {designs[np.argmax(scores)]}')

k = 1
percentile = 100
one_shot = scores[np.argmax(preds)][0]
normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
 


k = 16
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

k = 128
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')
 

k = 256
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 77.62721252441406
 The corresponding design is: [7.8898743e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.8368714e-01
 0.0000000e+00 0.0000000e+00 4.3004041e+00 3.4725893e-01 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 6.1534494e-02 0.0000000e+00 0.0000000e+00 1.0715234e+00
 0.0000000e+00 0.0000000e+00 2.4722518e-01 9.6047562e-01 0.0000000e+00
 0.0000000e+00 0.0000000e+00 2.7910426e-01 1.3624820e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 7.3111212e-01 1.0030539e+00 1.5582004e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.9704975e-01
 3.45277

In [20]:
augment = True
y_ubound = task.normalize_y(task.oracle.internal_dataset.y.max())[0]
if normalize_y:
    print('y_given before normalization:',  y_ubound[-1])
    y_given = torch.tensor(task.normalize_y(task.oracle.internal_dataset.y.max())[0])
    # y_given = torch.tensor(task.normalize_y(135)[0])
    # y_given = torch.tensor(y_ubound)
else:
    y_given = torch.tensor(task.oracle.internal_dataset.y.max())
 
print('y_given:', y_given)
if y_max.item() < y_given:
    y_max = y_given 
if  y_min.item() > y_given:
    y_min = y_given

if augment:
    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

 
# seed_list = [2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032]# , 2033, 2034
seed_list = [ 40, 41,42,   44, 45, 46, 47, 49] 
# seed_list = [ 39,40, 42, ] 

reported_results_1shot = []
reported_results_16shot = []
reported_results_128shot = []
reported_results_256shot = []

args.budget = 256
num_steps = 5
inference_scaling_bool = False
duplicated_time = 64
guidance_scals2= 100
description = 'nonresampling'

# y_given = torch.tensor([2])
for seed in seed_list:
    print(f'***********************seed{seed}*************************')
    print(f"Running ours method for {args.task} with seed {seed}")
    
    res = mango_eval(
        task,
        task_name=args.task,
        forwardmodel=forwardmodel,
        inverse_model=inverse_model,
        logging_dir=args.logging_dir,
        hidden_size=args.hidden_size,
        num_samples=args.budget,
        device=args.device,
        seed=seed,
        clip_dic=clip_dic,
        augment=augment,
        gen_condition=y_given,
        guidance=True,
        num_steps=num_steps,
        inference_scaling_bool=inference_scaling_bool,
        duplicated_time=duplicated_time,
        guidance_scals2=guidance_scals2,
        description=description
    )
 

    # designs = np.load(os.path.join(args.logging_dir, "solution.npy"))
    # preds = np.load(os.path.join(args.logging_dir, "predictions.npy"))
    # scores = np.load(os.path.join(args.logging_dir, "scores.npy"))
    y_unob_max = task.oracle.internal_dataset.y.max()
    y_unob_min = task.oracle.internal_dataset.y.min()

    designs = res['x']
    preds =   res['y_gen']
    # preds = res['y']
    scores = res['y_scores']

    # designs = designs.reshape(1, -1, designs.shape[-1]) 
    # preds = preds.reshape(1, -1, preds.shape[-1])
    # scores = scores.reshape(1, -1, scores.shape[-1])
    print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
    print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

    
    # task = design_bench.make(args.task)
    if task.is_normalized_y:
        best_training_score = task.denormalize_y(np.max(task.dataset.y))#[0]
    else:
        best_training_score = np.max(task.dataset.y)
    print("Training dataset:\n The best score during training is: ",best_training_score )
    oracle = np.max(scores)
    print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
    print(f' The corresponding design is: {designs[np.argmax(scores)]}')

    k = 1
    percentile = 100
    one_shot = scores[np.argmax(preds)][0]
    normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    reported_results_1shot.append(normalize_one_shot)

    k = 4
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 16
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 128
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_128shot.append(normalize_few_shot)

    k = 256
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_256shot.append(normalize_few_shot)
    
print(f'***********************OVERALL RESULTS*************************')
print(f"reported_results_1shot: {np.mean(reported_results_1shot):.4f} ± {np.std(reported_results_1shot):.4f}")
print(f"reported_results_16shot: {np.mean(reported_results_16shot):.4f} ± {np.std(reported_results_16shot):.4f}")
print(f"reported_results_128shot: {np.mean(reported_results_128shot):.4f} ± {np.std(reported_results_128shot):.4f}")
print(f"reported_results_256shot: {np.mean(reported_results_256shot):.4f} ± {np.std(reported_results_256shot):.4f}")

y_given before normalization: 8.168762
y_given: tensor([8.1688])
{'simple_clip': True, 'clip_min': tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9925]), 'clip_max': tensor([ 14.0000

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 107.51301574707031
 The corresponding design is: [0.         0.         0.         0.         0.         0.
 0.         0.5367809  0.02889207 0.         0.         0.
 0.         0.         0.         0.         0.22413312 0.
 0.         3.366913   0.7361209  0.         0.24500194 0.08340898
 0.         0.         0.         0.88102645 2.0898414  0.41970217
 0.6451425  0.         0.         0.         0.550823   0.
 1.3384619  3.3382568  0.47211957 0.         0.         0.09580793
 0.29779083 0.         0.5315542  0.         0.         0.
 0.         0.         0.39267975 0.         0.         0.
 0.         2.8604062  0.5326995  0.20646012 0.         0.3508859
 0.         0.         0.

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 103.76683044433594
 The corresponding design is: [0.28155965 0.         0.         0.         0.15470305 0.
 0.         0.769605   0.67896247 0.         0.         0.
 0.12759438 0.05636994 0.         0.         0.70436794 0.
 0.         2.5662134  0.         0.11597067 0.         0.45227075
 0.         0.14457637 0.         0.7597145  2.7065985  0.
 0.         0.         0.         0.         1.0635095  0.
 1.1289845  2.7877905  0.02938697 0.46767476 0.         0.
 0.         0.         0.         0.         0.12382913 0.
 0.91907483 1.2029139  0.01748721 0.14973679 0.06657001 0.
 0.55914515 2.0243597  0.21494979 0.76462865 0.         0.06371358
 0.         0.         0.11109376 0.    

...

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 96.46119689941406
 The corresponding design is: [0.         0.         0.         0.         0.         0.
 0.         1.0471485  0.16738346 0.         0.01374291 0.
 0.         0.         0.3836076  0.         0.         0.
 0.         1.6337394  0.         0.3106831  0.         0.2246012
 0.8502301  0.         0.         0.6561939  2.0988388  0.
 0.5244764  0.         0.         0.         0.         0.
 0.809602   1.0416722  0.         0.14918333 0.         0.
 0.614791   0.         0.         0.         0.49167308 0.
 0.         0.1853655  0.53172106 0.         0.         0.
 0.649931   2.1852465  0.         0.         0.47224227 0.05058566
 0.         1.3775338  0.         0.      

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 87.99046325683594
 The corresponding design is: [0.         0.         0.         0.         0.3721858  0.
 0.         1.197477   0.08650418 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.56959116 2.1367152  0.51316655 0.         0.         0.
 0.22635871 0.         0.         0.         2.142675   0.
 0.         0.         0.         0.         0.         0.
 1.1854995  1.6381751  0.         0.         0.         0.
 0.7897291  0.01069455 0.16510405 0.         0.34561467 0.
 0.         0.61988455 1.0627737  0.4009056  0.         0.
 0.34228927 0.9742782  0.         0.         0.13568108 0.
 0.         0.26674473 0.         0.         0.         0

/home/tzhouaq/anaconda3/envs/mango4soo/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
